In [1]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import timm
from tqdm import tqdm

In [2]:
# configs
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 64
num_workers = 4
image_size = 224

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

val_dir = '/scratch/bc3603/imagenet1k/val/imagenet-val'

val_dataset = datasets.ImageFolder(root=val_dir, transform=transform)
val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers
)

model_names = {
    'ConvNeXt-Tiny': 'convnext_tiny',
    'ViT-Small': 'vit_small_patch16_224',
    'ResNet50': 'resnet50',
}

/scratch/bc3603/my_env/lib/python3.11/site-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [3]:
for name, timm_name in model_names.items():
    model = timm.create_model(timm_name, pretrained=False)
    num_params = sum(p.numel() for p in model.parameters())
    print(f"{name}: {num_params / 1e6:.2f}M parameters")

ConvNeXt-Tiny: 28.59M parameters
ViT-Small: 22.05M parameters
ResNet50: 25.56M parameters


In [4]:
def evaluate(model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(val_loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return 100 * correct / total

In [3]:
results = {}

for name, timm_name in model_names.items():
    model = timm.create_model(timm_name, pretrained=True)
    model.to(device)

    acc = evaluate(model)
    results[name] = acc
    print(f"{name} Top-1 Accuracy: {acc:.2f}%")

print("\n--- Final Accuracy ---")
for name, acc in results.items():
    print(f"{name}: {acc:.2f}%")

100%|██████████| 782/782 [07:24<00:00,  1.76it/s]


ConvNeXt-Tiny Top-1 Accuracy: 84.00%


model.safetensors:   0%|          | 0.00/88.2M [00:00<?, ?B/s]

100%|██████████| 782/782 [07:20<00:00,  1.77it/s]


ViT-Small Top-1 Accuracy: 74.64%


100%|██████████| 782/782 [07:20<00:00,  1.78it/s]

ResNet50 Top-1 Accuracy: 80.12%

--- Final Accuracy ---
ConvNeXt-Tiny: 84.00%
ViT-Small: 74.64%
ResNet50: 80.12%


In [21]:
import sys
sys.path.append('/scratch/bc3603/MLfinal/tiny_vit_repo')

from models import tiny_vit  # model file is under tiny_vit_repo/models/tiny_vit.py
import torch

vit = tiny_vit.tiny_vit_21m_224(pretrained=False)

checkpoint = torch.load('/scratch/bc3603/MLfinal/tiny_vit_21m_22kto1k_distill.pth', map_location='cpu')
vit.load_state_dict(checkpoint['model'])

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
vit = vit.to(device)
vit.eval()

print("TinyViT loaded successfully.")

TinyViT loaded successfully.


In [24]:
def evaluate(model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(val_loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return 100 * correct / total
vit_acc = evaluate(vit)
print(f"TinyViT-21M Top-1 Accuracy: {vit_acc:.2f}%")

100%|██████████| 782/782 [07:28<00:00,  1.74it/s]

TinyViT-21M Top-1 Accuracy: 84.59%


In [31]:
import sys
import importlib

if '/scratch/bc3603/MLfinal/tiny_vit_repo' in sys.path:
    sys.path.remove('/scratch/bc3603/MLfinal/tiny_vit_repo')
sys.path.append('/scratch/bc3603/MLfinal/convnext_repo')
if 'models' in sys.modules:
    del sys.modules['models']

from models import convnext
import torch

convnext = convnext.convnext_tiny(num_classes=1000)

checkpoint = torch.load('/scratch/bc3603/MLfinal/convnext_tiny_22k_1k_224.pth', map_location='cpu')
convnext.load_state_dict(checkpoint['model'])

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
convnext = convnext.to(device)
convnext.eval()

print("ConvNeXt-Tiny loaded successfully.")

/scratch/bc3603/MLfinal/convnext_repo/models/convnext.py:158: UserWarning: Overwriting convnext_tiny in registry with models.convnext.convnext_tiny. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  @register_model
/scratch/bc3603/MLfinal/convnext_repo/models/convnext.py:167: UserWarning: Overwriting convnext_small in registry with models.convnext.convnext_small. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  @register_model
/scratch/bc3603/MLfinal/convnext_repo/models/convnext.py:176: UserWarning: Overwriting convnext_base in registry with models.convnext.convnext_base. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  @register_model
/scratch/bc3603/MLfinal/convnext_repo/models/convnext.py:185: UserWarning: Overwriting convnext_large in registry with models.convnext.convnext_large. This is

ConvNeXt-Tiny loaded successfully.


In [32]:
convnext_acc = evaluate(convnext)
print(f"ConvNeXt-Tiny Top-1 Accuracy: {convnext_acc:.2f}%")

100%|██████████| 782/782 [07:30<00:00,  1.74it/s]

ConvNeXt-Tiny Top-1 Accuracy: 82.47%


In [5]:
res_transform = transforms.Compose([
    transforms.Resize(448),
    transforms.CenterCrop(448),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5]
    ),
])
val_dataset = datasets.ImageFolder(root=val_dir, transform=res_transform)
val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers
)

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

resnet = timm.create_model(
    'hf-hub:timm/resnetv2_50x1_bit.goog_in21k_ft_in1k',
    pretrained=True
)

resnet = resnet.to(device)
resnet.eval()

print("ResNetV2-50x1 loaded successfully.")

ResNetV2-50x1 loaded successfully.


In [7]:
resnet_acc = evaluate(resnet)
print(f"ResNetV2-50x1 Top-1 Accuracy: {resnet_acc:.2f}%")

100%|██████████| 782/782 [12:04<00:00,  1.08it/s]

ResNetV2-50x1 Top-1 Accuracy: 80.34%
